In [2]:
import pandas as pd
import plotly.express as px
import plotly.graph_objects as go
from plotly.subplots import make_subplots
import numpy as np

In [4]:
state_abbrev = {
    'Alabama': 'AL', 'Alaska': 'AK', 'Arizona': 'AZ', 'Arkansas': 'AR', 'California': 'CA',
    'Colorado': 'CO', 'Connecticut': 'CT', 'Delaware': 'DE', 'Florida': 'FL', 'Georgia': 'GA',
    'Hawaii': 'HI', 'Idaho': 'ID', 'Illinois': 'IL', 'Indiana': 'IN', 'Iowa': 'IA',
    'Kansas': 'KS', 'Kentucky': 'KY', 'Louisiana': 'LA', 'Maine': 'ME', 'Maryland': 'MD',
    'Massachusetts': 'MA', 'Michigan': 'MI', 'Minnesota': 'MN', 'Mississippi': 'MS', 'Missouri': 'MO',
    'Montana': 'MT', 'Nebraska': 'NE', 'Nevada': 'NV', 'New Hampshire': 'NH', 'New Jersey': 'NJ',
    'New Mexico': 'NM', 'New York': 'NY', 'North Carolina': 'NC', 'North Dakota': 'ND', 'Ohio': 'OH',
    'Oklahoma': 'OK', 'Oregon': 'OR', 'Pennsylvania': 'PA', 'Rhode Island': 'RI', 'South Carolina': 'SC',
    'South Dakota': 'SD', 'Tennessee': 'TN', 'Texas': 'TX', 'Utah': 'UT', 'Vermont': 'VT',
    'Virginia': 'VA', 'Washington': 'WA', 'West Virginia': 'WV', 'Wisconsin': 'WI', 'Wyoming': 'WY'
}

In [24]:

import pandas as pd
import plotly.express as px
import plotly.graph_objects as go
from plotly.subplots import make_subplots
import numpy as np
import snowflake.connector
from snowflake.connector.pandas_tools import pd_writer

In [45]:
conn_params = {
    'user': 'ABHINAV',
    'password': 'Dortmund@1997!',
    'account': 'nba28627.east-us-2.azure',
    'warehouse': 'DBT_WH',
    'database': 'OPPORTUNITY_ANALYSIS',
    'schema': 'MARKET_ANALYSIS',
    'role': 'FIN_ROLE'
}

In [26]:
def connect_to_snowflake(params):
    try:
        conn = snowflake.connector.connect(
            user=params['user'],
            password=params['password'],
            account=params['account'],
            warehouse=params['warehouse'],
            database=params['database'],
            schema=params['schema'],
            role=params['role']  # Specify the role if needed
        )
        print("Connected to Snowflake successfully!")
        return conn
    except Exception as e:
        print(f"Error connecting to Snowflake: {e}")
        return None

In [27]:
def query_data(conn, industry='automotive'):
    query = f"""
    SELECT region, size_catagory as size_category, count(*) as count 
    FROM enhanced_companies 
    WHERE industry = '{industry}' 
    GROUP BY region, size_catagory
    """
    
    try:
        cursor = conn.cursor()
        cursor.execute(query)
        
        # Fetch column names
        col_names = [col[0] for col in cursor.description]
        
        # Fetch data
        data = cursor.fetchall()
        
        # Create DataFrame
        df = pd.DataFrame(data, columns=col_names)
        
        # Close cursor
        cursor.close()
        
        return df
    except Exception as e:
        print(f"Error executing query: {e}")
        return None

In [28]:
conn = connect_to_snowflake(conn_params)

if conn is None:
    print("Failed to connect to Snowflake. Exiting.")


Connected to Snowflake successfully!


In [29]:
df = query_data(conn)

In [30]:
df.head()

,REGION,SIZE_CATEGORY,COUNT
0,oklahoma,large,2
1,mississippi,large,2
2,connecticut,large,5
3,nebraska,large,4
4,north dakota,large,2


In [40]:
def create_heatmap(industry='automotive'):
    # Connect to Snowflake
    conn = connect_to_snowflake(conn_params)
    
    if conn is None:
        print("Failed to connect to Snowflake. Exiting.")
        return
    
    # Query data
    df = query_data(conn, industry)
    
    if df is None or df.empty:
        print(f"No data found for industry: {industry}")
        conn.close()
        return
    
    # Close connection
    conn.close()
    
    # Data preprocessing
    # Convert state names to title case for proper display
    print("rreached before title case")
    df['REGION'] = df['REGION'].str.title()
    print("rreached after title case")
    
    # Map state names to their abbreviations for the choropleth map
    state_abbrev = {
        'Alabama': 'AL', 'Alaska': 'AK', 'Arizona': 'AZ', 'Arkansas': 'AR', 'California': 'CA',
        'Colorado': 'CO', 'Connecticut': 'CT', 'Delaware': 'DE', 'Florida': 'FL', 'Georgia': 'GA',
        'Hawaii': 'HI', 'Idaho': 'ID', 'Illinois': 'IL', 'Indiana': 'IN', 'Iowa': 'IA',
        'Kansas': 'KS', 'Kentucky': 'KY', 'Louisiana': 'LA', 'Maine': 'ME', 'Maryland': 'MD',
        'Massachusetts': 'MA', 'Michigan': 'MI', 'Minnesota': 'MN', 'Mississippi': 'MS', 'Missouri': 'MO',
        'Montana': 'MT', 'Nebraska': 'NE', 'Nevada': 'NV', 'New Hampshire': 'NH', 'New Jersey': 'NJ',
        'New Mexico': 'NM', 'New York': 'NY', 'North Carolina': 'NC', 'North Dakota': 'ND', 'Ohio': 'OH',
        'Oklahoma': 'OK', 'Oregon': 'OR', 'Pennsylvania': 'PA', 'Rhode Island': 'RI', 'South Carolina': 'SC',
        'South Dakota': 'SD', 'Tennessee': 'TN', 'Texas': 'TX', 'Utah': 'UT', 'Vermont': 'VT',
        'Virginia': 'VA', 'Washington': 'WA', 'West Virginia': 'WV', 'Wisconsin': 'WI', 'Wyoming': 'WY'
    }
    
    # Handle any non-matching state names
    for region in df['REGION'].unique():
        if region not in state_abbrev:
            print(f"Warning: '{region}' is not a recognized US state name")
    
    # Add state abbreviations to the dataframe
    df['state_code'] = df['REGION'].map(state_abbrev)
    
    # Create a total count by state for the main heatmap
    state_totals = df.groupby('REGION')['COUNT'].sum().reset_index()
    state_totals['state_code'] = state_totals['REGION'].map(state_abbrev)
    
    # Define size categories order for consistent visualization
    size_order = ['1-10', '11-50', '51-200', '201-500', '501-1000', '1001-5000', '5001-10000', '10000+']
    
    # Create pivot table
    df['SIZE_CATEGORY'] = pd.Categorical(df['SIZE_CATEGORY'], categories=size_order, ordered=True)
    
    # Create a pivot table for size categories distribution
    size_pivot = df.pivot_table(
        values='COUNT', 
        index='REGION', 
        columns='SIZE_CATEGORY',
        aggfunc='sum',
        fill_value=0
    )
    
    # Normalize the data to show distribution percentages within each state
    size_pivot_normalized = size_pivot.div(size_pivot.sum(axis=1), axis=0) * 100
    
    # Add state codes to the pivot data
    size_pivot_normalized['state_code'] = size_pivot_normalized.index.map(state_abbrev)
    
    # Create the main choropleth map
    fig = make_subplots(
        rows=1, cols=1,
        specs=[[{"type": "choropleth"}]]
    )
    
    # Add the main choropleth map showing total company count by state
    fig.add_trace(
        go.Choropleth(
            locations=state_totals['state_code'],
            z=state_totals['COUNT'],
            locationmode='USA-states',
            colorscale='Viridis',
            colorbar_title="Total Companies",
            name=f"Total {industry.title()} Companies",
            marker_line_color='white',
            marker_line_width=0.5,
            hovertemplate='<b>%{location}</b><br>' +
                            'Total Companies: %{z}<br>' +
                            '<extra></extra>'
        ),
        row=1, col=1
    )
# Create a secondary visualization for the size distribution
    # We'll use a bar chart that appears on hover
    size_data = []
    for state in state_totals['REGION']:
        if state in size_pivot.index:
            for size_cat in size_order:
                if size_cat in size_pivot.columns:
                    value = size_pivot.loc[state, size_cat] if size_cat in size_pivot.columns else 0
                    pct = size_pivot_normalized.loc[state, size_cat] if size_cat in size_pivot_normalized.columns else 0
                    
                    # Add hover data
                    fig.add_trace(
                        go.Scatter(
                            x=[state_abbrev.get(state, "")],
                            y=[0],
                            mode='markers',
                            marker=dict(size=0, color='rgba(0,0,0,0)'),
                            hoverinfo='text',
                            hovertemplate=f'<b>{state}</b><br>' +
                                         f'Size: {size_cat}<br>' +
                                         f'Count: {value}<br>' +
                                         f'Percentage: {pct:.1f}%<br>' +
                                         '<extra></extra>',
                            showlegend=False
                        )
                    )
    
    # Update the layout
    fig.update_layout(
        title_text=f'US {industry.title()} Industry Distribution by State',
        title_x=0.5,
        geo=dict(
            scope='usa',
            projection=go.layout.geo.Projection(type='albers usa'),
            showlakes=True,
            lakecolor='rgb(255, 255, 255)'
        ),
        height=600,
        width=950,
        margin=dict(l=0, r=0, t=50, b=0),
        showlegend=False
    )
    
    # Add an annotation explaining the visualization
    fig.add_annotation(
        x=0.5,
        y=-0.1,
        xref='paper',
        yref='paper',
        text='Hover over states to see size category distribution',
        showarrow=False,
        font=dict(size=12)
    )
    
    # Create a second visualization for size categories in top states
    # Get top 5 states by company count
    top_states = state_totals.sort_values('COUNT', ascending=False).head(5)['REGION'].tolist()
    
    # Create subplots for the pie charts
    fig2 = make_subplots(
        rows=1, cols=min(5, len(top_states)),
        specs=[[{'type':'domain'} for _ in range(min(5, len(top_states)))]],
        subplot_titles=[f"{state}" for state in top_states]
    )
    
    # Add pie charts for each top state
    for i, state in enumerate(top_states):
        state_data = df[df['REGION'] == state]
        pie_data = state_data.groupby('SIZE_CATEGORY')['COUNT'].sum().reset_index()
        
        fig2.add_trace(
            go.Pie(
                labels=pie_data['SIZE_CATEGORY'],
                values=pie_data['COUNT'],
                name=state,
                marker_colors=px.colors.sequential.Viridis[:len(pie_data)],
                textinfo='percent',
                hoverinfo='label+percent+value',
                hovertemplate='<b>%{label}</b><br>' +
                             'Count: %{value}<br>' +
                             'Percentage: %{percent}<br>' +
                             '<extra></extra>'
            ),
            row=1, col=i+1
        )
    
    # Update layout for the second figure
    fig2.update_layout(
        title_text=f'Size Distribution in Top {len(top_states)} {industry.title()} Industry States',
        height=400,
        width=950
    )

    return fig, fig2

In [41]:
fig1, fig2 = create_heatmap(industry='automotive')

Connected to Snowflake successfully!
rreached before title case
rreached after title case


C:\Users\yohan\AppData\Local\Temp\ipykernel_20296\2068319353.py:59: FutureWarning:

The default value of observed=False is deprecated and will change to observed=True in a future version of pandas. Specify observed=False to silence this warning and retain the current behavior

C:\Users\yohan\AppData\Local\Temp\ipykernel_20296\2068319353.py:164: FutureWarning:

The default of observed=False is deprecated and will be changed to True in a future version of pandas. Pass observed=False to retain current behavior or observed=True to adopt the future default and silence this warning.

C:\Users\yohan\AppData\Local\Temp\ipykernel_20296\2068319353.py:164: FutureWarning:

The default of observed=False is deprecated and will be changed to True in a future version of pandas. Pass observed=False to retain current behavior or observed=True to adopt the future default and silence this warning.

C:\Users\yohan\AppData\Local\Temp\ipykernel_20296\2068319353.py:164: FutureWarning:

The default of observed

In [42]:
fig1.show()

In [43]:
fig2.show()

## Table updation with YFINANCE

In [44]:
import pandas as pd
import snowflake.connector
import yfinance as yf
import numpy as np
import datetime

In [46]:
def get_company_data():
    """Fetch company data with ticker symbols from Snowflake"""
    conn = connect_to_snowflake(conn_params)
    if conn is None:
        return None
    
    try:
        query = """
        SELECT * FROM LARGE_COMPANIES_WITH_TICKER
        WHERE T_SYMBOL IS NOT NULL
        """
        
        df = pd.read_sql(query, conn)
        conn.close()
        print(f"Retrieved {len(df)} companies with ticker symbols")
        return df
    except Exception as e:
        print(f"Error fetching company data: {e}")
        if conn:
            conn.close()
        return None 

In [50]:
def get_company_data():
    """Fetch company data with ticker symbols from Snowflake"""
    conn = connect_to_snowflake(conn_params)
    if conn is None:
        return None
    
    try:
        query = """
        SELECT * FROM LARGE_COMPANIES_WITH_TICKER
        WHERE T_SYMBOL IS NOT NULL
        """
        
        df = pd.read_sql(query, conn)
        conn.close()
        print(f"Retrieved {len(df)} companies with ticker symbols")
        return df
    except Exception as e:
        print(f"Error fetching company data: {e}")
        if conn:
            conn.close()
        return None

def get_yfinance_metrics(ticker_list):
    """Get relevant financial metrics for a list of tickers"""
    # Filter out None values
    ticker_list = [t for t in ticker_list if t]
    
    if not ticker_list:
        return pd.DataFrame()
    
    # Create a dictionary to store results
    results = {}
    
    # Process tickers in batches to avoid rate limiting
    batch_size = 20
    for i in range(0, len(ticker_list), batch_size):
        batch = ticker_list[i:i+batch_size]
        # print(f"Processing batch {i//batch_size + 1}/{(len(ticker_list) + batch_size - 1)//batch_size}")
        
        # Process each ticker in the batch
        for ticker in batch:
            try:
                # print(f"Processing ticker: {ticker}")
                # Create ticker object
                ticker_obj = yf.Ticker(ticker)
                
                # Get basic info
                info = ticker_obj.info
                
                # Get historical data for 1-year return calculation
                hist = ticker_obj.history(period="1y")
                
                # Calculate metrics if we have historical data
                if len(hist) > 0:
                    # Calculate return
                    start_price = hist['Close'].iloc[0]
                    current_price = hist['Close'].iloc[-1]
                    yearly_return = ((current_price - start_price) / start_price) * 100
                    
                    # Calculate volatility
                    daily_returns = hist['Close'].pct_change().dropna()
                    volatility = daily_returns.std() * 100 * np.sqrt(252)  # Annualized
                else:
                    yearly_return = None
                    volatility = None
                
                # Store the results
                results[ticker] = {
                    'ticker': ticker,
                    'current_price': hist['Close'].iloc[-1] if len(hist) > 0 else None,
                    'market_cap': info.get('marketCap', None),
                    'pe_ratio': info.get('trailingPE', None),
                    'forward_pe': info.get('forwardPE', None),
                    'price_to_book': info.get('priceToBook', None),
                    'beta': info.get('beta', None),
                    'dividend_yield': info.get('dividendYield', 0) * 100 if info.get('dividendYield') else 0,
                    'yearly_return': yearly_return,
                    'volatility': volatility,
                    'sector': info.get('sector', None),
                    'industry_yf': info.get('industry', None),
                    'company_name_yf': info.get('longName', None),
                    'revenue_growth': info.get('revenueGrowth', None),
                    'profit_margins': info.get('profitMargins', None),
                    'return_on_equity': info.get('returnOnEquity', None),
                    'debt_to_equity': info.get('debtToEquity', None),
                    'avg_volume': info.get('averageVolume', None),
                    'target_mean_price': info.get('targetMeanPrice', None),
                    'analyst_rating': info.get('recommendationMean', None),
                    'last_updated': datetime.datetime.now().strftime('%Y-%m-%d')
                }
                
            except Exception as e:
                print(f"Error processing ticker {ticker}: {e}")
                # Add the ticker to results with None values to maintain the record
                results[ticker] = {
                    'ticker': ticker,
                    'current_price': None,
                    'market_cap': None,
                    'pe_ratio': None,
                    'forward_pe': None,
                    'price_to_book': None,
                    'beta': None,
                    'dividend_yield': None,
                    'yearly_return': None,
                    'volatility': None,
                    'sector': None,
                    'industry_yf': None,
                    'company_name_yf': None,
                    'revenue_growth': None,
                    'profit_margins': None,
                    'return_on_equity': None,
                    'debt_to_equity': None,
                    'avg_volume': None,
                    'target_mean_price': None,
                    'analyst_rating': None,
                    'last_updated': datetime.datetime.now().strftime('%Y-%m-%d')
                }
    
    # Convert results to DataFrame
    metrics_df = pd.DataFrame.from_dict(results, orient='index')
    return metrics_df

def enhance_company_data_with_yfinance():
    """Main function to enhance company data with yfinance metrics"""
    # Get company data from Snowflake
    company_df = get_company_data()
    
    if company_df is None or company_df.empty:
        print("No company data available. Exiting.")
        return
    
    # Extract unique ticker symbols
    ticker_symbols = company_df['T_SYMBOL'].unique().tolist()
    print(f"Found {len(ticker_symbols)} unique ticker symbols")
    
    # Get metrics from yfinance
    metrics_df = get_yfinance_metrics(ticker_symbols)
    
    if metrics_df.empty:
        print("No metrics data retrieved. Exiting.")
        return
    
    # Merge the data
    enhanced_df = pd.merge(
        company_df,
        metrics_df,
        left_on='T_SYMBOL',
        right_on='ticker',
        how='left'
    )
    
    print(f"Enhanced dataset has {enhanced_df.shape[1]} columns and {enhanced_df.shape[0]} rows")
    
    # Write the enhanced data back to Snowflake
    # save_to_snowflake(enhanced_df)
    
    # Also save locally as CSV for backup
    enhanced_df.to_csv('enhanced_company_data.csv', index=False)
    print("Data saved to enhanced_company_data.csv")
    
    return enhanced_df

In [51]:
en_df = enhance_company_data_with_yfinance()


Connected to Snowflake successfully!


C:\Users\yohan\AppData\Local\Temp\ipykernel_20296\3518601462.py:13: UserWarning:

pandas only supports SQLAlchemy connectable (engine/connection) or database string URI or sqlite3 DBAPI2 connection. Other DBAPI2 objects are not tested. Please consider using SQLAlchemy.



Retrieved 894 companies with ticker symbols
Found 737 unique ticker symbols


404 Client Error: Not Found for url: https://query2.finance.yahoo.com/v10/finance/quoteSummary/ANG$B?modules=financialData%2CquoteType%2CdefaultKeyStatistics%2CassetProfile%2CsummaryDetail&corsDomain=finance.yahoo.com&formatted=false&symbol=ANG%24B&crumb=fhSyczRYun3


Error processing ticker ANG$B: 'NoneType' object has no attribute 'update'


...

Error processing ticker SOUL.U: 'NoneType' object has no attribute 'update'


404 Client Error: Not Found for url: https://query2.finance.yahoo.com/v10/finance/quoteSummary/CLDT$A?modules=financialData%2CquoteType%2CdefaultKeyStatistics%2CassetProfile%2CsummaryDetail&corsDomain=finance.yahoo.com&formatted=false&symbol=CLDT%24A&crumb=fhSyczRYun3


Error processing ticker CLDT$A: 'NoneType' object has no attribute 'update'


404 Client Error: Not Found for url: https://query2.finance.yahoo.com/v10/finance/quoteSummary/SCE$J?modules=financialData%2CquoteType%2CdefaultKeyStatistics%2CassetProfile%2CsummaryDetail&corsDomain=finance.yahoo.com&formatted=false&symbol=SCE%24J&crumb=fhSyczRYun3


Error processing ticker SCE$J: 'NoneType' object has no attribute 'update'
Enhanced dataset has 36 columns and 894 rows
Data saved to enhanced_company_data.csv


In [53]:
en_df.tail()

,ID,INDUSTRY,COMPANY_NAME,SIZE,FOUNDED,REGION,COUNTRY,LOCALITY,WEBSITE,LINKEDIN_URL,...,industry_yf,company_name_yf,revenue_growth,profit_margins,return_on_equity,debt_to_equity,avg_volume,target_mean_price,analyst_rating,last_updated
889,e64MvwCCRwULJn3dpfCPoQLkY6Vi,financial services,the hartford,10001+,1810,connecticut,united states,hartford,thehartford.com,linkedin.com/company/the-hartford,...,Insurance - Property & Casualty,"The Hartford Insurance Group, Inc.",0.074,0.11713,0.19582,27.427,1805885.0,130.0000,2.31579,2025-04-11
890,V6kvOeBij5MTbwYCGf4QEQRTXQVv,consumer goods,the hershey company,10001+,1894,pennsylvania,united states,hershey,thehersheycompany.com,linkedin.com/company/the-hershey-company,...,Confectioners,The Hershey Company,0.087,0.19828,0.50404,115.545,2017479.0,165.2224,3.00000,2025-04-11
891,BDN04pBEqr4T7NpAvKYB1AhPCbUa,hospitality,the marcus corporation,10001+,1935,wisconsin,united states,milwaukee,marcuscorp.com,linkedin.com/company/marcus-corporation,...,Entertainment,The Marcus Corporation,0.162,-0.01120,-0.01664,75.857,213667.0,26.0000,NaN,2025-04-11
892,Gk1yEjJBlLCQswdhHUrtmALGZHXD,research,thermo fisher scientific,10001+,1956,massachusetts,united states,waltham,thermofisher.com,linkedin.com/company/thermo-fisher-scientific,...,Diagnostics & Research,Thermo Fisher Scientific Inc.,0.047,0.14774,0.13134,66.099,2118143.0,643.0782,1.61290,2025-04-11
893,6wkAxWFvrOdCEQESsu5yyQXbPQyR,oil & energy,thermon,1001-5000,1954,texas,united states,san marcos,thermon.com,linkedin.com/company/thermon,...,Specialty Industrial Machinery,"Thermon Group Holdings, Inc.",-0.015,0.09481,0.09753,34.152,206272.0,35.0000,NaN,2025-04-11


In [ ]:
import pandas as pd
import finnhub

#list of available exchanges

df=pd.read_html("https://docs.google.com/spreadsheets/d/1I3pBxjfXB056-g_JYf_6o3Rns3BV2kMGG1nCatb91ls/edit#gid=0")
df1=df[0]
exc=df1.loc[:,"A"].dropna()

exclist=[]
for i in exc:
    exclist.append(str(i))
exclist=exclist[1:] #take out "name" from the list

#retrieve tickers from every exchange available
tickers=[]
finnhub_client = finnhub.Client(api_key="c46qn9iad3iagvmhdk7g")
for exchange in exclist:
    listofdicts=finnhub_client.stock_symbols(exchange)
    for dicts in listofdicts:
        tickers.append(dicts['symbol'])

print("You just got a list of %s tickers worldwide." % len(tickers), tickers)

In [2]:
import time
import pandas as pd
import yfinance as yf
from get_all_tickers.get_tickers import get_tickers

tickers_list = get_tickers()

tickers_list = tickers_list[:50]

# Initialize an empty list to store the data.
data = []

for ticker in tickers_list:
    try:
        stock = yf.Ticker(ticker)
        info = stock.info  # Retrieve the info dictionary from yfinance
        
        # Extract the company name and market capitalization.
        company_name = info.get('longName', 'N/A')
        market_cap = info.get('marketCap', 'N/A')
        
        # Append the data as a dictionary.
        data.append({
            'Ticker': ticker,
            'Company Name': company_name,
            'Market Cap': market_cap
        })
        
        # Pause briefly to respect rate limits.
        time.sleep(0.1)
    except Exception as e:
        print(f"Error retrieving data for {ticker}: {e}")
        continue

# Create a DataFrame from the list of dictionaries.


c:\Users\yohan\anaconda3\envs\final_proj_1\Lib\site-packages\get_all_tickers\get_tickers.py:95: SyntaxWarning: invalid escape sequence '\.'
  df = df[~df['Symbol'].str.contains("\.|\^")]
c:\Users\yohan\anaconda3\envs\final_proj_1\Lib\site-packages\get_all_tickers\get_tickers.py:140: SyntaxWarning: invalid escape sequence '\.'
  df_filtered = df[~df['Symbol'].str.contains("\.|\^")]
c:\Users\yohan\anaconda3\envs\final_proj_1\Lib\site-packages\get_all_tickers\get_tickers.py:147: SyntaxWarning: invalid escape sequence '\.'
  df = df[~df['Symbol'].str.contains("\.|\^")]


ParserError: Error tokenizing data. C error: Expected 1 fields in line 9, saw 2


In [3]:
from pytickersymbols import PyTickerSymbols

stock_data = PyTickerSymbols()
german_stocks = stock_data.get_stocks_by_index('DAX')
uk_stocks = stock_data.get_stocks_by_index('FTSE 100')

print(list(uk_stocks))

[{'name': '3i Group PLC', 'symbol': 'III', 'country': 'United Kingdom', 'indices': ['FTSE 100'], 'industries': ['Banking & Investment Services', 'Investment Banking & Investment Services', 'Financials', 'Investment Management & Fund Operators'], 'symbols': [{'yahoo': 'IGQ5.F', 'google': 'FRA:IGQ5', 'currency': 'EUR'}, {'yahoo': 'TGOPY', 'google': 'OTCMKTS:TGOPY', 'currency': 'USD'}, {'yahoo': 'III.L', 'google': 'LON:III', 'currency': 'GBP'}, {'yahoo': 'IGQ.F', 'google': 'FRA:IGQ', 'currency': 'EUR'}, {'yahoo': 'TGOPF', 'google': 'OTCMKTS:TGOPF', 'currency': 'USD'}], 'metadata': {'founded': 1945, 'employees': 244}, 'isins': ['GB00B1YW4409'], 'akas': []}, {'name': 'Admiral Group PLC', 'symbol': 'ADM', 'country': 'United Kingdom', 'indices': ['FTSE 100'], 'industries': ['Insurance', 'Property & Casualty Insurance', 'Financials'], 'symbols': [{'yahoo': 'FLN.F', 'google': 'FRA:FLN', 'currency': 'EUR'}, {'yahoo': 'ADM.L', 'google': 'LON:ADM', 'currency': 'GBP'}, {'yahoo': 'AMIGY', 'google': 

In [5]:
from pytickersymbols import PyTickerSymbols

# Initialize the PyTickerSymbols object
stock_data = PyTickerSymbols()

# Retrieve stocks for each index
sp500_stocks = list(stock_data.get_stocks_by_index('S&P 500'))
dowjones_stocks = list(stock_data.get_stocks_by_index('DOW JONES'))
nasdaq100_stocks = list(stock_data.get_stocks_by_index('NASDAQ 100'))

# Combine all stocks into a single list
us_stocks = sp500_stocks + dowjones_stocks + nasdaq100_stocks

# Extract unique ticker symbols
tickers = list({stock['symbol'] for stock in us_stocks})

In [35]:
import ftplib
import pandas as pd
from io import StringIO

def fetch_us_listed_tickers():
    ftp = ftplib.FTP('ftp.nasdaqtrader.com')
    ftp.login()
    ftp.encoding = 'utf-8'
    ftp.cwd('SymbolDirectory')

    files = ["nasdaqlisted.txt", "otherlisted.txt"]
    all_dfs = []

    for file in files:
        buffer = []
        ftp.retrlines(f"RETR {file}", buffer.append)
        content = "\n".join(buffer)
        
        df = pd.read_csv(StringIO(content), sep="|")
        df = df[:-1]  # remove footer row
        all_dfs.append(df)

    ftp.quit()

    # Combine and return a unified DataFrame with standardized columns
    combined_df = pd.concat(all_dfs, ignore_index=True)
    combined_df.rename(columns={'Symbol': 'ticker', 'Security Name': 'company_name'}, inplace=True)
    return combined_df[['ticker', 'company_name']].drop_duplicates()

In [24]:
import yfinance as yf
import time

def enrich_with_yfinance(df: pd.DataFrame, sleep_time: float = 0.5) -> pd.DataFrame:
    """
    Adds 'yf_company_name' and 'market_cap' columns to the DataFrame using yfinance.
    Original 'company_name' from NASDAQ remains untouched for reference.
    """
    enriched_data = []

    for _, row in df.iterrows():
        ticker = row['ticker']
        try:
            stock = yf.Ticker(ticker)
            info = stock.info
            yf_name = info.get('longName', 'N/A')
            market_cap = info.get('marketCap', 'N/A')
        except Exception as e:
            print(f"Error for {ticker}: {e}")
            yf_name = "Error"
            market_cap = None
        
        enriched_data.append({
            'ticker': ticker,
            'original_name': row['company_name'],
            'yf_company_name': yf_name,
            'market_cap': market_cap
        })

        time.sleep(sleep_time)  # avoid rate limiting

    return pd.DataFrame(enriched_data)

In [36]:
tickers_df = fetch_us_listed_tickers()


In [38]:
tickers_df = tickers_df.drop_duplicates(subset=['ticker'])  # Remove duplicates based on ticker

In [39]:
tickers_df.head()

,ticker,company_name
0,AACB,Artius II Acquisition Inc. - Class A Ordinary ...
1,AACBR,Artius II Acquisition Inc. - Rights
2,AACBU,Artius II Acquisition Inc. - Units
3,AACG,ATA Creativity Global - American Depositary Sh...
4,AADR,AdvisorShares Dorsey Wright ADR ETF


In [26]:
enriched_df = enrich_with_yfinance(tickers_df)

500 Server Error: Internal Server Error for url: https://query2.finance.yahoo.com/v10/finance/quoteSummary/LUCYW?modules=financialData%2CquoteType%2CdefaultKeyStatistics%2CassetProfile%2CsummaryDetail&corsDomain=finance.yahoo.com&formatted=false&symbol=LUCYW&crumb=fhSyczRYun3


Error for LUCYW: 'NoneType' object has no attribute 'update'
Error for nan: 'float' object has no attribute 'upper'


404 Client Error: Not Found for url: https://query2.finance.yahoo.com/v10/finance/quoteSummary/ZAZZT?modules=financialData%2CquoteType%2CdefaultKeyStatistics%2CassetProfile%2CsummaryDetail&corsDomain=finance.yahoo.com&formatted=false&symbol=ZAZZT&crumb=fhSyczRYun3


Error for ZAZZT: 'NoneType' object has no attribute 'update'


404 Client Error: Not Found for url: https://query2.finance.yahoo.com/v10/finance/quoteSummary/ZBZZT?modules=financialData%2CquoteType%2CdefaultKeyStatistics%2CassetProfile%2CsummaryDetail&corsDomain=finance.yahoo.com&formatted=false&symbol=ZBZZT&crumb=fhSyczRYun3


...

404 Client Error: Not Found for url: https://query2.finance.yahoo.com/v10/finance/quoteSummary/ZCZZT?modules=financialData%2CquoteType%2CdefaultKeyStatistics%2CassetProfile%2CsummaryDetail&corsDomain=finance.yahoo.com&formatted=false&symbol=ZCZZT&crumb=fhSyczRYun3


Error for ZCZZT: 'NoneType' object has no attribute 'update'


404 Client Error: Not Found for url: https://query2.finance.yahoo.com/v10/finance/quoteSummary/ZXYZ.A?modules=financialData%2CquoteType%2CdefaultKeyStatistics%2CassetProfile%2CsummaryDetail&corsDomain=finance.yahoo.com&formatted=false&symbol=ZXYZ.A&crumb=fhSyczRYun3


Error for ZXYZ.A: 'NoneType' object has no attribute 'update'
Error for nan: 'float' object has no attribute 'upper'
Error for nan: 'float' object has no attribute 'upper'
Error for nan: 'float' object has no attribute 'upper'
Error for nan: 'float' object has no attribute 'upper'
Error for nan: 'float' object has no attribute 'upper'
Error for nan: 'float' object has no attribute 'upper'
Error for nan: 'float' object has no attribute 'upper'
Error for nan: 'float' object has no attribute 'upper'
Error for nan: 'float' object has no attribute 'upper'
Error for nan: 'float' object has no attribute 'upper'
Error for nan: 'float' object has no attribute 'upper'
Error for nan: 'float' object has no attribute 'upper'
Error for nan: 'float' object has no attribute 'upper'
Error for nan: 'float' object has no attribute 'upper'
Error for nan: 'float' object has no attribute 'upper'
Error for nan: 'float' object has no attribute 'upper'
Error for nan: 'float' object has no attribute 'upper'
Err

KeyboardInterrupt: 

In [22]:
def filter_valid_companies(df: pd.DataFrame):
    """
    Filters the DataFrame to keep only rows with valid company names and market cap.
    """
    return df[
        (df['yf_company_name'].notna()) &
        (df['yf_company_name'] != 'N/A') &
        (df['yf_company_name'] != 'Error') &
        (df['market_cap'] != 'N/A')
    ].reset_index(drop=True)

In [ ]:
enriched_df

100

In [23]:
filtered_df = filter_valid_companies(enriched_df)
filtered_df.head()

,ticker,original_name,yf_company_name,market_cap
0,AACB,Artius II Acquisition Inc. - Class A Ordinary ...,Artius II Acquisition Inc. Class A Ordinary Sh...,248854880
1,AACBU,Artius II Acquisition Inc. - Units,Artius II Acquisition Inc. Units,253260512
2,AACG,ATA Creativity Global - American Depositary Sh...,ATA Creativity Global,30211798
3,AAL,"American Airlines Group, Inc. - Common Stock",American Airlines Group Inc.,6358759936
4,AAME,Atlantic American Corporation - Common Stock,Atlantic American Corporation,26931038


In [ ]:
filtered_df.to_csv('filtered_us_companies.csv', index=False)